# Active Patterns examples

## Reference material

### Deep Dive into Active Patterns
Paul Blasucci @pblasucci

Video of talk
https://www.youtube.com/watch?v=Q5KO-UDx5eA

Slides and examples
https://github.com/pblasucci/DeepDiveAP

### FSharp for Fun and Profit

https://fsharpforfunandprofit.com/posts/convenience-active-patterns/

### .NET docs

https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/active-patterns

## Introduction

### Active pattern of one choice.

In [ ]:
// Active pattern of one choice.
let (|ItemCount|) input = input |> List.length

let applyDiscount orderlines =
  match orderlines with
  | ItemCount 4 -> printfn "Apply 5% discount"
  | ItemCount 8 -> printfn "Apply 10% discount"
  | ItemCount i -> printfn "No discount for %i items" i

applyDiscount [1;2]
applyDiscount [1;2;3;4]
applyDiscount [1;2;3;4;5;6;7;8]


Stopped due to error


Error: input.fsx (8,28)-(8,54) typecheck error The type 'int -> unit' does not match the type 'unit'

### Active Patterns with multiple choices

In [ ]:
let (|Even|Odd|) input = if input % 2 = 0 then Even else Odd

let TestNumber input =
  match input with
  | Even -> printfn "%d is even" input
  | Odd -> printfn "%d is odd" input

TestNumber 7
TestNumber 11
TestNumber 32

7 is odd
11 is odd
32 is even


### Partial Active Patterns

Active patterns that do not always produce a value are called partial active patterns.

* To define a partial active pattern, you use a wildcard character (_) at the end of the list of patterns.



In [ ]:
// Partial active pattern
let (|Jeff|_|) (input:string) = if input.Equals("Jeff") then Some () else None

let isJeff name = 
  match name with
  | Jeff -> printfn "My name Jeff"
  | _ -> printfn "Not Jeff"

isJeff "Geoff"

Not Jeff


## Parsing of strings

In [ ]:
open System.Text.RegularExpressions

let (|Int|_|) str =
  match System.Int32.TryParse(str:string) with
  | (true,int) -> Some(int)
  | _ -> None

let (|Bool|_|) str =
  match System.Boolean.TryParse(str:string) with
  | (true,bool) -> Some(bool)
  | _ -> None

let (|ParseRegex|_|) regex str =
  let m = Regex(regex).Match(str)
  if m.Success then Some ()
  else None

let (|ParseRegexGroup|_|) regex str =
  let m = Regex(regex).Match(str)
  if m.Success then Some (List.tail [ for x in m.Groups -> x.Value ])
  else None

### Int and bool Example
https://fsharpforfunandprofit.com/posts/convenience-active-patterns/

In [ ]:
// create a function to call the patterns
let testParse str =
  match str with
  | Int i -> printfn "The value is an int '%i'" i
  | Bool b -> printfn "The value is a bool '%b'" b
  | _ -> printfn "The value '%s' is something else" str

// test
testParse "12"
testParse "true"
testParse "abc"

The value is an int '12'
The value is a bool 'true'
The value 'abc' is something else


In [ ]:
let (|IsValidEmail|_|) input =
  match input with
  | ParseRegex ".*?@(.*)" -> Some ()
  | _ -> None

let requireEmail value =
  match value with
  | IsValidEmail -> Ok value
  | _ -> Error "Email is not in the correct format"


requireEmail "foo@bar.com"

ResultValue,ErrorValue
foo@bar.com,<null>


In [ ]:
type EmailParts = {
  Username: string
  Domain: string
  TopLevelDomain: string
}

let emailParts str =
  let emailPattern = @"([a-zA-Z0-9+._-]+)@([a-zA-Z0-9._-]+)\.([a-zA-Z0-9_-]+)"
  match str with
  | ParseRegexGroup emailPattern [u ; d; tld] -> 
      {Username = u; Domain = d; TopLevelDomain = tld} 
  | _ -> failwith "not found"

let parts = emailParts "foo@bar.com"

printfn $"User:\t{parts.Username}"
printfn $"Domain:\t{parts.Domain}"
printfn $"TLD:\t{parts.TopLevelDomain}"


User:	foo
Domain:	bar
TLD:	com


## Changing shape of data
Using active patterns on let-bindings

In [ ]:
let (|RGB|) (col : System.Drawing.Color) =
  ( col.R, col.G, col.B )

let (|HSB|) (col : System.Drawing.Color) =
  ( col.GetHue(), col.GetSaturation(), col.GetBrightness() )

let (RGB redRgb) = System.Drawing.Color.Red
let (HSB redHsb) = System.Drawing.Color.Red

printfn $"{redRgb}"
printfn $"{redHsb}"

(255, 0, 0)
(0, 1, 0.5)


In [ ]:
type CustomerDetails = { 
  name : Name
  title : string
  email : string
  mobilePhone : string
  dateOfBirth : string
  address : Address
  } 
  
  and Name = {
    firstName : string
    middleName : string option
    lastName : string 
  }
  
  and Address = {
    addressLine1 : string
    city : string
    state : string
    postcode : string
    countryCode : string
  }

let profile =
  { name = { firstName = "Jane"; middleName = None; lastName = "Citizen" }
    title = "Ms"
    email = $"EXT01@company.com.au"
    mobilePhone = "+61412345678"
    dateOfBirth = "1990-12-20"
    address = {  
      addressLine1 = "1 Wonder Street"
      city = "Wonderland"
      state = "NSW"
      postcode = "2000"
      countryCode = "AU" } }

let (|EmailWithAddress|) (p:CustomerDetails) = (p.email, p.address)

let (EmailWithAddress (email, address)) = profile

printfn $"Email:\n{email}"
printfn $"Address:\n{address}"

Email:
EXT01@company.com.au
Address:
{ addressLine1 = "1 Wonder Street"
  city = "Wonderland"
  state = "NSW"
  postcode = "2000"
  countryCode = "AU" }


## Taking pattern matching further

In [ ]:
let (|Between|_|) (x: int) (y: int) (input: int) =
  if input >= x && input <= y
  then Some()
  else None

let isLeapYear (year: int) = DateTime.IsLeapYear(year)

let parseDate year month day =
  if year <= 0 then
    Error "Year cannot be 0 or less"
  else
    match month, day with
    | 2, Between 1 29 when isLeapYear year -> Ok (DateTime(year, month, day))
    | 2, Between 1 28 when not (isLeapYear year) -> Ok (DateTime(year, month, day))
    | (1 | 3 | 5 | 7 | 8 | 10 | 12), Between 1 31 -> Ok (DateTime(year, month, day))
    | (4 | 6 | 9 | 11), Between 1 30 -> Ok (DateTime(year, month, day))
    | _ -> Error "Date invalid"

parseDate 2021 02 30

ResultValue,ErrorValue
0001-01-01 00:00:00Z,Date invalid


In [ ]:
let (|February|_|) (month: int) =
  if month = 2 then Some () else None

let (|Thirtydays|_|) (month: int) =
  match month with
  | (4 | 6 | 9 | 11) -> Some ()
  | _ -> None

let (|ThirtyOnedays|_|) (month: int) =
  match month with
  | (1 | 3 | 5 | 7 | 8 | 10 | 12)-> Some ()
  | _ -> None

let parseDate year month day =
  if year <= 0 then
    Error "Year cannot be 0 or less"
  else
    match month, day with
    | February, Between 1 29 when isLeapYear year -> Ok (DateTime(year, month, day))
    | February, Between 1 28 when not (isLeapYear year) -> Ok (DateTime(year, month, day))
    | ThirtyOnedays, Between 1 31 -> Ok (DateTime(year, month, day))
    | Thirtydays, Between 1 30 -> Ok (DateTime(year, month, day))
    | _ -> Error "Date invalid"

parseDate 2021 02 30

ResultValue,ErrorValue
0001-01-01 00:00:00Z,Date invalid
